#### Multiprocessing in Python
The multiprocessing module in Python allows you to create processes that can run in parallel, making it possible to fully utilize multiple CPU cores. Unlike threading, multiprocessing creates separate memory spaces for each process, avoiding the limitations of the Global Interpreter Lock (GIL).

1. Multiprocessing Module Basics
Creating Parallel Processes
You can use multiprocessing.Process to create and run parallel processes.

In [ ]:
from multiprocessing import Process
import time

def func(id):
    time.sleep(1)
    print(f"{id} function is running")

if __name__ == "__main__":
    start = time.time()
    for i in range(10):
        process = []
        p = Process(target=func, args=(i,))
        process.append(p)
        p.start()

    for p in process:
        p.join()
    print("all work done in : ",time.time()-start)

2. Inter-Process Communication
Since processes in multiprocessing do not share memory, inter-process communication is necessary for sharing data between them. Python provides two primary methods: Queues and Pipes.

<h3>Using Queues</h3>
multiprocessing.Queue allows safe sharing of data between processes.

In [1]:
from multiprocessing import Process, Queue

def producer(queue):
    for i in range(5):
        queue.put(f"Item {i}")
    queue.put(None)  # Sentinel to indicate completion

def consumer(queue):
    while True:
        item = queue.get()
        if item is None:
            break
        print(f"Consumed {item}")

if __name__ == "__main__":
    queue = Queue()
    p1 = Process(target=producer, args=(queue,))
    p2 = Process(target=consumer, args=(queue,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()


#### Using Pipes
multiprocessing.Pipe provides a two-way communication channel between processes.

In [ ]:
from multiprocessing import Process, Pipe

def sender(conn):
    conn.send("Hello from Sender!")
    conn.close()

def receiver(conn):
    message = conn.recv()
    print(f"Received: {message}")

if __name__ == "__main__":
    parent_conn, child_conn = Pipe()
    p1 = Process(target=sender, args=(child_conn,))
    p2 = Process(target=receiver, args=(parent_conn,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()


3. Shared Memory
To share data between processes without the need for inter-process communication, multiprocessing provides shared memory objects.

#### Using multiprocessing.Value
A Value is a single shared variable that can be updated by multiple processes.

In [ ]:
from multiprocessing import Process, Value
import time

def increment(shared_value):
    for _ in range(1000):
        shared_value.value += 1

if __name__ == "__main__":
    shared_value = Value('i', 0)  # 'i' indicates integer type
    processes = [Process(target=increment, args=(shared_value,)) for _ in range(4)]
    
    for p in processes:
        p.start()
    for p in processes:
        p.join()

    print(f"Final Value: {shared_value.value}")


#### Using multiprocessing.Array
An Array is a shared array of values that can be updated by multiple processes.

In [ ]:
from multiprocessing import Process, Array

def square_elements(shared_array, k):
    for i in range(len(shared_array)):
        shared_array[i] = shared_array[i] ** k

if __name__ == "__main__":
    shared_array = Array('i', [1, 2, 3, 4, 5])  # 'i' indicates integer type
    p1 = Process(target=square_elements, args=(shared_array,2))
    p2 = Process(target=square_elements, args=(shared_array,3))

    p1.start()
    p2.start()
    p1.join()
    p2.join()

    print("Updated Array:", list(shared_array))


### Advanced Topics in Multiprocessing
1. Using multiprocessing.Pool for Task Parallelism<br>
The Pool class in multiprocessing simplifies the parallel execution of a function across a collection of inputs. It manages a pool of worker processes and distributes tasks among them.

In [ ]:
from multiprocessing import Pool

def square(x):
    return x * x

if __name__ == "__main__":
    data = [1, 2, 3, 4, 5]
    with Pool(processes=4) as pool:  # Create a pool with 4 workers
        results = pool.map(square, data)  # Apply the `square` function to each element
    print(results)


#### Key Methods of Pool:
map(func, iterable): Applies a function to all elements in an iterable (like map in Python).<br>
apply(func, args): Executes a function with the given arguments in one worker process.<br>
apply_async(func, args): Like apply, but non-blocking; returns a multiprocessing.AsyncResult.<br>
starmap(func, iterable): Similar to map, but expects arguments to be passed as tuples.<br>

2. Process Synchronization
In multiprocessing, shared resources can cause conflicts if accessed simultaneously. Synchronization primitives like Locks, Semaphores, and Events ensure safe access.

#### Using a Lock
A Lock prevents multiple processes from accessing a critical section simultaneously.

In [ ]:
from multiprocessing import Process, Lock

def critical_section(lock, task_id):
    with lock:  # Automatically acquires and releases the lock
        print(f"Task {task_id} entering critical section")
        print(f"Task {task_id} leaving critical section")

if __name__ == "__main__":
    lock = Lock()
    processes = [Process(target=critical_section, args=(lock, i)) for i in range(4)]

    for p in processes:
        p.start()
    for p in processes:
        p.join()


#### Other Synchronization Tools:
Semaphore: Limits the number of processes that can access a resource.
Event: Allows communication between processes by setting and clearing flags.

3. Debugging Multiprocessing Applications
Debugging multiprocessing can be challenging because each process runs independently. Here are some tips and tools to make it easier:

#### Tips:
Log Messages: Use the logging module instead of print statements, as print output can get jumbled in multiprocessing. <br>
Isolate Issues: Run the same code in a single process to debug logic errors.<br>
Multiprocessing Manager: Use multiprocessing.Manager to debug shared states between processes.<br>

4. Managing Shared States with multiprocessing.Manager  <br>
A Manager provides a way to share Python objects like lists, dictionaries, and namespaces between processes. These objects are managed by a server process.

In [ ]:
from multiprocessing import Process, Manager

def update_shared_list(shared_list, value):
    shared_list.append(value)

if __name__ == "__main__":
    with Manager() as manager:
        shared_list = manager.list()  # Shared list
        processes = [Process(target=update_shared_list, args=(shared_list, i)) for i in range(5)]

        for p in processes:
            p.start()
        for p in processes:
            p.join()

        print("Updated shared list:", list(shared_list))


5. Terminating Processes <br>
Processes can be terminated prematurely using the terminate method.

In [ ]:
from multiprocessing import Process
import time

def infinite_loop():
    while True:
        print("Running...")
        time.sleep(1)

if __name__ == "__main__":
    p = Process(target=infinite_loop)
    p.start()
    time.sleep(5)  # Let the process run for 5 seconds
    p.terminate()  # Terminate the process
    print("Process terminated")


6. Handling Exceptions in Multiprocessing
Exceptions raised in child processes do not propagate to the parent process. You can catch them by using an AsyncResult.

In [ ]:
from multiprocessing import Pool

def divide(x, y):
    return x / y

if __name__ == "__main__":
    with Pool(processes=4) as pool:
        results = pool.apply_async(divide, args=(1, 0))
        try:
            print(results.get())  # This will raise a ZeroDivisionError
        except ZeroDivisionError as e:
            print(f"Caught exception: {e}")


#### Summary
The multiprocessing module is a powerful tool for parallelism in Python. <br>
Here's what we covered:<br>

Process Creation: Using multiprocessing.Process.<br>
Inter-Process Communication: With Queues and Pipes.<br>
Shared Memory: Using multiprocessing.Value, multiprocessing.Array, and Manager.list<br>
Synchronization: Ensuring thread-safe operations with Locks and Semaphores.<br>
Advanced Features: Using Pools, debugging tips, exception handling, and process termination.<br>